In [42]:
# https://noumenon-th.net/programming/2016/04/27/logisticregression/
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
%matplotlib inline

In [43]:
train_df = pd.read_csv('./input/train.csv')
test_df = pd.read_csv('./input/test.csv')

In [44]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [45]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [46]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [47]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [48]:
#利用しない変数は削除
train_df = train_df.drop(['Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)
test_df = test_df.drop(['Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)

In [49]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,male,22.0
1,2,1,1,female,38.0
2,3,1,3,female,26.0
3,4,1,1,female,35.0
4,5,0,3,male,35.0


In [50]:
test_df.head()

,PassengerId,Pclass,Sex,Age
0,892,3,male,34.5
1,893,3,female,47.0
2,894,2,male,62.0
3,895,3,male,27.0
4,896,3,female,22.0


## 欠損値の補完

In [73]:
#年齢の欠損値を男女の平均年齢で補間
age_train_mean = train_df.groupby('Sex').Age.mean()
 
def fage(x):
    if x.Sex == 'male':
        return round(age_train_mean['male'])
    if x.Sex == 'female':
        return round(age_train_mean['female'])

train_df.Age.fillna(train_df[train_df.Age.isnull()].apply(fage,axis=1),inplace=True)


TypeError: "value" parameter must be a scalar, dict or Series, but you passed a "DataFrame"

In [52]:
age_test_mean = test_df.groupby('Sex').Age.mean()
 
def fage(x):
    if x.Sex == 'male':
        return round(age_test_mean['male'])
    if x.Sex == 'female':
        return round(age_test_mean['female'])

test_df.Age.fillna(test_df[test_df.Age.isnull()].apply(fage,axis=1),inplace=True)

## ダミー変数

In [53]:
#クロス集計
sex_ct = pd.crosstab(train_df['Sex'], train_df['Survived'])
sex_ct

Survived,0,1
Sex,,
female,81,233
male,468,109


In [54]:
#Femaleカラムを追加し、Sex要素のmale/femaleを1/0に変換して、要素として追加する
train_df['Female'] = train_df['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
test_df['Female'] = test_df['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
 
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,Female
0,1,0,3,male,22.0,0
1,2,1,1,female,38.0,1
2,3,1,3,female,26.0,1
3,4,1,1,female,35.0,1
4,5,0,3,male,35.0,0


In [55]:
#クロス集計
pclass_ct = pd.crosstab(train_df['Pclass'], train_df['Survived'])
pclass_ct

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [56]:
#Pclassをダミー変数で分ける
pclass_train_df  = pd.get_dummies(train_df['Pclass'],prefix='Class')
pclass_test_df  = pd.get_dummies(test_df['Pclass'],prefix='Class')
 
pclass_train_df.head()

,Class_1,Class_2,Class_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [57]:
#Class_3を削除
pclass_train_df = pclass_train_df.drop(['Class_3'], axis=1)
pclass_test_df = pclass_test_df.drop(['Class_3'], axis=1)
 
#Class_1,Class_2カラムを追加
train_df = train_df.join(pclass_train_df)
test_df = test_df.join(pclass_test_df)

In [58]:
#訓練用
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,Female,Class_1,Class_2
0,1,0,3,male,22.0,0,0,0
1,2,1,1,female,38.0,1,1,0
2,3,1,3,female,26.0,1,0,0
3,4,1,1,female,35.0,1,1,0
4,5,0,3,male,35.0,0,0,0


In [59]:
#テスト用
test_df.head()

,PassengerId,Pclass,Sex,Age,Female,Class_1,Class_2
0,892,3,male,34.5,0,0,0
1,893,3,female,47.0,1,0,0
2,894,2,male,62.0,0,0,1
3,895,3,male,27.0,0,0,0
4,896,3,female,22.0,1,0,0


## モデル生成と予測

In [60]:
X = train_df.drop(['PassengerId','Survived','Pclass','Sex'],axis=1)
y = train_df.Survived
 
#モデルの生成
clf = LogisticRegression()
 
#学習
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [61]:
#学習したモデルの精度
clf.score(X,y)

#モデルに伴う生存率の予測値
#predict_y = clf.predict(X)
 
#実際の値と予測値の比率
#accuracy_score(y, predict_y)

0.8013468013468014

In [62]:
#変数名とその係数を格納するデータフレーム
coeff_df = DataFrame([X.columns, clf.coef_[0]]).T
coeff_df

,0,1
0,Age,-0.0335092
1,Female,2.46126
2,Class_1,2.16662
3,Class_2,1.08844


In [63]:
#テストデータから生存者を予測
X1 = test_df.drop(['PassengerId','Pclass','Sex'],axis=1)
test_predict = clf.predict(X1)

In [64]:
submission = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived':np.array(test_predict)})
submission.to_csv('submission.csv', index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
